### Code Implementation

In this section, we implement the **PageRank algorithm**. The algorithm computes the PageRank scores based on the transition matrix derived from the adjacency matrix of the graph. We also incorporate the **teleportation factor** to ensure stability in the algorithm, especially for handling dangling nodes and ensuring convergence.

The code below:
- Normalizes the adjacency matrix to create the **transition matrix**.
- Uses the **power iteration method** to iteratively compute the PageRank scores until convergence.
- Runs the algorithm for different **damping factors** to analyze the effect of the teleportation probability on the stability and convergence of the rankings.


In [ ]:
import numpy as np

# Define the transition matrix M (adjacency matrix representing forward links)
M = np.array([
    [0, 0, 1],
    [0.5, 0, 0],
    [0.5, 1, 0]  ])

# Number of pages (nodes)
n = M.shape[0]

# Define the teleportation probability (p)
p = 0.85

# Initialize the PageRank vector
pagerank = np.ones(n) / n

# Create the teleportation vector q (assuming each page has a different probability of being visited)
# q can represent any custom distribution based on how likely each page is to be visited.
q = np.array([0.333333, 0.333333, 0.333333])

# Number of iterations to run
iterations = 10

# Calculate the PageRank iteratively
for i in range(iterations):
    # Compute the PageRank for the next iteration
    pagerank = p * np.dot(M, pagerank) + (1 - p) * q  # Using teleportation vector q

    # Print the PageRank vector for each iteration
    print("Iteration {}: {}".format(i + 1, pagerank))


Iteration 1: [0.33333328 0.19166662 0.47499995]
Iteration 2: [0.45374991 0.1916666  0.35458322]
Iteration 3: [0.35139569 0.24284366 0.40576027]
Iteration 4: [0.39489618 0.19934312 0.40576023]
Iteration 5: [0.39489614 0.21783083 0.38727247]
Iteration 6: [0.37918155 0.21783081 0.40298701]
Iteration 7: [0.39253891 0.21115211 0.3963083 ]
Iteration 8: [0.386862   0.21682899 0.39630828]
Iteration 9: [0.38686199 0.2144163  0.39872094]
Iteration 10: [0.38891275 0.2144163  0.39667015]


In [ ]:
import networkx as nx

# Create the directed graph
G = nx.DiGraph()

# Add edges as per tutorial example
G.add_edges_from([('A', 'B'), ('A', 'C'), ('B', 'C'), ('C', 'A')])

# Set damping factor (alpha)
alpha = 0.85

# Calculate PageRank
pagerank_scores = nx.pagerank(G, alpha=alpha)

# Print the results
print(pagerank_scores)


{'A': 0.387789442707259, 'B': 0.21481051315058508, 'C': 0.3974000441421556}


### HITS Algorithm Implementation

In this section, we compute the **HITS (Hyperlink-Induced Topic Search)** algorithm for the same graph used in the **PageRank** example. The HITS algorithm computes two values for each node:
- **Hub score**: A measure of how many good authority pages a node points to.
- **Authority score**: A measure of how many good hubs point to a node.

We will use the **`nx.hits()`** function from **NetworkX** to compute these scores for the graph.


In [ ]:
import networkx as nx

# Create the directed graph (same graph used for PageRank)
G = nx.DiGraph()

# Add edges as per the tutorial example
G.add_edges_from([('A', 'B'), ('A', 'C'), ('B', 'C'), ('C', 'A')])

# Calculate HITS hub and authority values using NetworkX's hits function
hubs, authorities = nx.hits(G, max_iter=10, tol=1e-8, normalized=True)

# Print the results for hub and authority values
print("Hub values:")
for node, hub_value in hubs.items():
    print(f"Node {node}: {hub_value:.4f}")

print("\nAuthority values:")
for node, authority_value in authorities.items():
    print(f"Node {node}: {authority_value:.4f}")


Hub values:
Node A: 0.6180
Node B: 0.3820
Node C: -0.0000

Authority values:
Node A: -0.0000
Node B: 0.3820
Node C: 0.6180


### Randomized HITS Algorithm

In this section, we implement the **Randomized HITS** algorithm, which is designed to stabilize the traditional **HITS (Hyperlink-Induced Topic Search)** algorithm by introducing a **random teleportation** component. The random surfer alternates between following **forward** and **backward** links and occasionally **teleports** to a randomly chosen page with a given probability.

#### Key Features:
- **Forward and Backward Links**: The surfer alternates between following **out-links** (forward) and **in-links** (backward) at each time step.
- **Teleportation**: With a probability \( S \), the surfer jumps to a random node, ensuring stability and preventing the walk from getting stuck in isolated nodes (dangling nodes).
- **Stability**: The addition of teleportation reduces the impact of small perturbations in the graph structure, making the algorithm more stable than traditional **HITS**.

In [ ]:
import networkx as nx
import random

def randomized_hits(G, S=0.15, max_steps=1000, start_node=None, tol=1e-8):
    """
    Implements the Randomized HITS algorithm, computing the hub and authority scores
    with teleportation, following the alternating random walk between forward and backward links.

    Parameters:
    - G : NetworkX directed graph
    - S : float, teleportation probability (default=0.15)
    - max_steps : int, maximum number of steps in the random walk (default=1000)
    - start_node : node, the starting node for the random walk (default=None, randomly chosen)

    Returns:
    - hubs : dictionary, hub scores for each node
    - authorities : dictionary, authority scores for each node
    """

    # Initialize the hub and authority scores
    n = len(G)
    if start_node is None:
        start_node = random.choice(list(G.nodes))  # Randomly choose a starting node

    current_node = start_node
    visited_nodes = [current_node]

    # Initialize hub and authority scores (all to 1 initially)
    hubs = {node: 1.0 for node in G}
    authorities = {node: 1.0 for node in G}

    # Teleportation vector (uniform distribution)
    q = {node: 1.0 / n for node in G}

    # Perform the random walk and update hub and authority scores
    for step in range(1, max_steps + 1):
        hubs_last = hubs.copy()
        authorities_last = authorities.copy()

        # Toss the coin: with probability S, teleport to a random page
        if random.random() < S:
            current_node = random.choice(list(G.nodes))
        else:
            # Alternating between forward and backward links
            if step % 2 == 1:  # Odd step: follow a forward link (outgoing)
                if G.out_degree(current_node) > 0:
                    current_node = random.choice(list(G.neighbors(current_node)))
            else:  # Even step: follow a backward link (incoming)
                if G.in_degree(current_node) > 0:
                    current_node = random.choice(list(G.predecessors(current_node)))

        visited_nodes.append(current_node)

        # Update hub and authority scores
        for node in hubs:
            # Update hub scores (based on incoming links)
            hubs[node] = (1 - S) * sum(authorities.get(neighbor, 0)
            for neighbor in G.neighbors(node)) + S * q.get(node, 0)

        for node in authorities:
            # Update authority scores (based on outgoing links)
            authorities[node] = (1 - S) * sum(hubs.get(neighbor, 0)
            for neighbor in G.predecessors(node)) + S * q.get(node, 0)

        # Normalize hub and authority scores
        hub_sum = sum(hubs.values())
        authority_sum = sum(authorities.values())
        if hub_sum != 0:
            hubs = {node: score / hub_sum for node, score in hubs.items()}
        if authority_sum != 0:
            authorities = {node: score / authority_sum for node, score in authorities.items()}

        # Check for convergence (L1 norm)
        err_hub = sum(abs(hubs[node] - hubs_last[node]) for node in hubs)
        err_authority = sum(abs(authorities[node] - authorities_last[node]) for node in authorities)

        if err_hub < tol and err_authority < tol:
            break

    return hubs, authorities

# Example usage
G = nx.DiGraph()

# Add edges as per the tutorial example (same graph used for PageRank)
G.add_edges_from([('A', 'B'), ('A', 'C'), ('B', 'C'), ('C', 'A')])

# Compute the Randomized HITS hubs and authorities
hubs, authorities = randomized_hits(G, S=0.15, max_steps=50, start_node='A')

# Print the results for hub and authority values
print("Hub values:")
for node, hub_value in hubs.items():
    print(f"Node {node}: {hub_value:.4f}")

print("\nAuthority values:")
for node, authority_value in authorities.items():
    print(f"Node {node}: {authority_value:.4f}")


Hub values:
Node A: 0.5673
Node B: 0.3617
Node C: 0.0711

Authority values:
Node A: 0.0656
Node B: 0.3599
Node C: 0.5745


### Subspace HITS Algorithm

This section implements the **Subspace HITS** algorithm to compute the hub and authority scores for a given directed graph. The algorithm improves the stability of the original **HITS** by considering multiple eigenvectors instead of relying on just the principal eigenvector.

**Key Steps:**
1. **Adjacency Matrix**: We start by constructing the adjacency matrix \( A \) of the graph.
2. **Hub and Authority Matrices**: The hub matrix \( H = A \times A^T \) and authority matrix \( A_{\text{matrix}} = A^T \times A \) are computed.
3. **Eigenvalue Decomposition**: Eigenvalue decomposition is performed to extract the top \( q \) eigenvectors corresponding to the largest eigenvalues from both matrices.
4. **Subspace Calculation**: The top \( q \) eigenvectors are aggregated to form a subspace that is used to calculate the hub and authority scores.
5. **Normalization**: The scores are normalized to ensure that they sum to 1.

This approach captures the broader structure of the graph and improves stability, especially in dynamic graphs with perturbations.


In [ ]:
import networkx as nx
import numpy as np

def subspace_hits(G, q=None, tol=1e-6):
    """
    Implements the Subspace HITS algorithm to compute hub and authority scores.
    Uses all eigenvectors by default (q = n), but can use a subset if q is specified,
    and weights the eigenvectors by lambda^2 (eigenvalue squared) to give more importance
    to larger eigenvalues.

    Parameters:
    - G : NetworkX directed graph
    - q : number of top eigenvectors to consider for the subspace (default is None, i.e., use all eigenvectors)
    - tol : tolerance for convergence (default=1e-6)

    Returns:
    - hubs : dictionary with hub scores for each node
    - authorities : dictionary with authority scores for each node
    """

    # Step 1: Create the adjacency matrix (A)
    A = nx.to_numpy_array(G, dtype=float)

    # Step 2: Compute the hub (A * A.T) and authority (A.T * A) matrices
    H = np.dot(A, A.T)  # Hub matrix (A * A.T)
    A_matrix = np.dot(A.T, A)  # Authority matrix (A.T * A)

    # Step 3: Perform eigenvalue decomposition to get eigenvectors and eigenvalues
    eigvals_H, eigvecs_H = np.linalg.eig(H)
    eigvals_A, eigvecs_A = np.linalg.eig(A_matrix)

    # Step 4: If q is not provided, use all eigenvectors (q = n)
    if q is None:
        q = len(G.nodes)

    # Step 5: Apply lambda^2 to the eigenvalues (larger eigenvalues get more weight)
    eigvals_H_squared = np.power(np.abs(eigvals_H), 2)  # Squared eigenvalues for hubs
    eigvals_A_squared = np.power(np.abs(eigvals_A), 2)  # Squared eigenvalues for authorities

    # Step 6: Select the top q eigenvectors based on the sorted eigenvalues
    sorted_indices_H = np.argsort(eigvals_H)[::-1][:q]  # Top q eigenvectors for hubs
    sorted_indices_A = np.argsort(eigvals_A)[::-1][:q]  # Top q eigenvectors for authorities

    # Step 7: Apply the eigenvalues to the selected eigenvectors
    weighted_hub_subspace = np.dot(eigvecs_H[:, sorted_indices_H], np.diag(eigvals_H_squared[sorted_indices_H]))  # Apply weight to hub eigenvectors
    weighted_authority_subspace = np.dot(eigvecs_A[:, sorted_indices_A], np.diag(eigvals_A_squared[sorted_indices_A]))  # Apply weight to authority eigenvectors

    # Step 8: Calculate the hub and authority scores from the weighted subspaces
    hubs = np.sum(weighted_hub_subspace, axis=1)  # Sum across the weighted subspaces for hubs
    authorities = np.sum(weighted_authority_subspace, axis=1)  # Sum across the weighted subspaces for authorities

    # Step 9: Normalize the results (to keep the scores between 0 and 1)
    hubs /= np.sum(hubs)
    authorities /= np.sum(authorities)

    # Step 10: Return the hub and authority scores
    hubs_dict = dict(zip(G.nodes(), hubs))
    authorities_dict = dict(zip(G.nodes(), authorities))

    return hubs_dict, authorities_dict

# Example usage
G = nx.DiGraph()
G.add_edges_from([('A', 'B'), ('A', 'C'), ('B', 'C'), ('C', 'A')])

# Compute Subspace HITS hubs and authorities using all eigenvectors weighted by eigenvalues squared
hubs, authorities = subspace_hits(G)

# Print results
print("Hub values:")
for node, hub_value in hubs.items():
    print(f"Node {node}: {hub_value:.4f}")

print("\nAuthority values:")
for node, authority_value in authorities.items():
    print(f"Node {node}: {authority_value:.4f}")


Hub values:
Node A: 0.5490
Node B: 0.3556
Node C: 0.0954

Authority values:
Node A: 0.0954
Node B: 0.3556
Node C: 0.5490


### Small Graph Experiment: Comparison of Link Analysis Algorithms

In this experiment, we will apply the four link analysis algorithms—**PageRank**, **HITS**, **Randomized HITS**, and **Subspace HITS**—to a small directed graph. The graph consists of 3 nodes and 4 edges, representing web pages and their hyperlinks. We will compute the **hub** and **authority** scores for each node using these algorithms and compare the results.


Each algorithm will be tested on the following:
1. **PageRank**: Using the damping factor \( \alpha = 0.85 \), we will compute the **PageRank** scores for the nodes.
2. **HITS**: Using the **HITS** function from NetworkX, we will calculate the **hub** and **authority** scores for each node.
3. **Randomized HITS**: We will implement the **Randomized HITS** algorithm, which incorporates teleportation and alternates between following forward and backward links.
4. **Subspace HITS**: We will compute the **Subspace HITS** scores by taking the top \( q \) eigenvectors from the hub and authority matrices.

The goal of this experiment is to compare the rankings of the nodes across all four algorithms and analyze how stable the results are, particularly under small graph perturbations.

### Expected Outcomes:
- **PageRank** should provide a global ranking based on the link structure.
- **HITS** will differentiate nodes based on their role as hubs or authorities.
- **Randomized HITS** should offer more stability but may yield similar results to **HITS**.
- **Subspace HITS** should provide stability by using a subspace of multiple eigenvectors.




In [ ]:
import networkx as nx
import random
import numpy as np

# Define the graph structure
def create_graph():
    G = nx.DiGraph()
    G.add_edges_from([
        (1, 2), (1, 3),
        (2, 1), (2, 5),
        (3, 2), (3, 8),
        (4, 3),
        (5, 4), (5, 8),
        (6, 4), (6, 5),
        (7, 4), (7, 6),
        (8, 1), (8, 4), (8, 7)
    ])
    return G

# Function to calculate L1 norm (sum of absolute differences) between two dictionaries
def calculate_l1_norm(dict1, dict2):
    return sum(abs(dict1[node] - dict2.get(node, 0)) for node in dict1)

# Function to perform stability test on different algorithms
def stability_test(G, algorithm_function, *args, **kwargs):
    l1_norms = []  # Store the L1 norms for all iterations

    # Repeat the perturbation and scoring process 5 times
    for i in range(5):
        # Step 1: Compute initial scores
        if algorithm_function == test_pagerank:
            scores_initial = algorithm_function(G, *args, **kwargs)
            scores_perturbed = scores_initial
        else:
            hubs_initial, authorities_initial = algorithm_function(G, *args, **kwargs)
            scores_initial = (hubs_initial, authorities_initial)

        # Perturbation: Remove 1 random edge (temporary removal)
        edges = list(G.edges)  # Get the current state of edges in the graph
        if len(edges) > 1:
            edge_to_remove = random.choice(edges)

            # Remove edge from the graph temporarily for this iteration
            G.remove_edge(*edge_to_remove)
        else:
            continue  # Skip if there are not enough edges

        # Step 2: Recompute scores after perturbation
        if algorithm_function == test_pagerank:
            scores_perturbed = algorithm_function(G, *args, **kwargs)
        else:
            hubs_perturbed, authorities_perturbed = algorithm_function(G, *args, **kwargs)
            scores_perturbed = (hubs_perturbed, authorities_perturbed)

        # Step 3: Calculate L1 norm to compare stability
        if algorithm_function == test_pagerank:
            norm = calculate_l1_norm(scores_initial, scores_perturbed)
        else:
            hub_norm = calculate_l1_norm(scores_initial[0], scores_perturbed[0])  # Compare hub scores
            authority_norm = calculate_l1_norm(scores_initial[1], scores_perturbed[1])  # Compare authority scores
            norm = (hub_norm, authority_norm)

        # Add the norm to the list for averaging later
        l1_norms.append(norm)

        # Restore the edge after each test (to make sure it is available for future tests)
        G.add_edge(*edge_to_remove)

    # Return the average L1 norm across all iterations
    if algorithm_function == test_pagerank:
        return np.mean(l1_norms)
    else:
        # For HITS, return a tuple of (average hub norm, average authority norm)
        hub_norms = [x[0] for x in l1_norms]
        authority_norms = [x[1] for x in l1_norms]
        return np.mean(hub_norms), np.mean(authority_norms)


# Function to test PageRank stability
def test_pagerank(G, alpha=0.85):
    return nx.pagerank(G, alpha=alpha)


# Function to test HITS stability
def test_hits(G, max_iter=100, tol=1e-8):
    hubs, authorities = nx.hits(G, max_iter=max_iter, tol=tol, normalized=True)
    return hubs, authorities

# Function to test Randomized HITS stability
def test_randomized_hits(G, S=0.15, max_steps=1000, start_node=None, tol=1e-8):
    return randomized_hits(G, S, max_steps, start_node, tol)

# Function to test Subspace HITS stability
def test_subspace_hits(G, q=2, max_iter=100, tol=1e-6):
    return subspace_hits(G, q, tol)


# Example of testing the modified stability function for all 4 algorithms
G = create_graph()

# Test each algorithm for stability
print("Testing PageRank stability...")
pagerank_score_norm = stability_test(G, test_pagerank)
print(f"PageRank L1 norm: {pagerank_score_norm}")

print("\nTesting HITS stability...")
hits_hub_norm, hits_authority_norm = stability_test(G, test_hits)
print(f"HITS L1 norm for hub scores: {hits_hub_norm}")
print(f"HITS L1 norm for authority scores: {hits_authority_norm}")

print("\nTesting Randomized HITS stability...")
rand_hits_hub_norm, rand_hits_authority_norm = stability_test(G, test_randomized_hits)
print(f"Randomized HITS L1 norm for hub scores: {rand_hits_hub_norm}")
print(f"Randomized HITS L1 norm for authority scores: {rand_hits_authority_norm}")


print("\nTesting Subspace HITS stability...")
subspace_hits_hub_norm, subspace_hits_authority_norm = stability_test(G, test_subspace_hits)
print(f"Subspace HITS L1 norm for hub scores: {subspace_hits_hub_norm}")
print(f"Subspace HITS L1 norm for authority scores: {subspace_hits_authority_norm}")

Testing PageRank stability...
PageRank L1 norm: 0.12035134848944255

Testing HITS stability...
HITS L1 norm for hub scores: 0.2482128318847831
HITS L1 norm for authority scores: 0.24455893469770001

Testing Randomized HITS stability...
Randomized HITS L1 norm for hub scores: 0.15864950553976595
Randomized HITS L1 norm for authority scores: 0.12936497057634005

Testing Subspace HITS stability...
Subspace HITS L1 norm for hub scores: 0.2202328318847831
Subspace HITS L1 norm for authority scores: 0.2322589346977


## **Displaying Results for the Full Graph (Small Experiment)**

We will now print the top 10 node scores for each algorithm using the full graph from the small experiment. This will provide us with the baseline scores before any perturbation.


In [ ]:
# Function to print top 10 nodes with their scores in descending order
def print_top_10_scores(scores):
    top_10 = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:10]
    for node, score in top_10:
        print(f"Node {node}: {score:.4f}")

def create_graph():
    G = nx.DiGraph()
    G.add_edges_from([
        (1, 2), (1, 3),
        (2, 1), (2, 5),
        (3, 2), (3, 8),
        (4, 3),
        (5, 4), (5, 8),
        (6, 4), (6, 5),
        (7, 4), (7, 6),
        (8, 1), (8, 4), (8, 7)
    ])
    return G

G = create_graph()

# Re-run the four algorithms on the full small graph

print("\nRe-running PageRank on full graph...")
pagerank_scores = test_pagerank(G)
print("Top 10 PageRank scores:")
print_top_10_scores(pagerank_scores)

print("\nRe-running HITS on full graph...")
hits_hub_scores, hits_authority_scores = test_hits(G)
print("Top 10 HITS hub scores:")
print_top_10_scores(hits_hub_scores)
print("Top 10 HITS authority scores:")
print_top_10_scores(hits_authority_scores)

print("\nRe-running Randomized HITS on full graph...")
rand_hits_hub_scores, rand_hits_authority_scores = test_randomized_hits(G)
print("Top 10 Randomized HITS hub scores:")
print_top_10_scores(rand_hits_hub_scores)
print("Top 10 Randomized HITS authority scores:")
print_top_10_scores(rand_hits_authority_scores)

print("\nRe-running Subspace HITS on full graph...")
subspace_hits_hub_scores, subspace_hits_authority_scores = test_subspace_hits(G)
print("Top 10 Subspace HITS hub scores:")
print_top_10_scores(subspace_hits_hub_scores)
print("Top 10 Subspace HITS authority scores:")
print_top_10_scores(subspace_hits_authority_scores)



Re-running PageRank on full graph...
Top 10 PageRank scores:
Node 3: 0.2015
Node 2: 0.1590
Node 4: 0.1507
Node 8: 0.1492
Node 1: 0.1286
Node 5: 0.1053
Node 7: 0.0610
Node 6: 0.0447

Re-running HITS on full graph...
Top 10 HITS hub scores:
Node 8: 0.2517
Node 6: 0.1980
Node 5: 0.1834
Node 7: 0.1719
Node 2: 0.1221
Node 3: 0.0540
Node 1: 0.0156
Node 4: 0.0033
Top 10 HITS authority scores:
Node 4: 0.3580
Node 1: 0.1663
Node 5: 0.1423
Node 7: 0.1120
Node 8: 0.1056
Node 6: 0.0765
Node 2: 0.0310
Node 3: 0.0084

Re-running Randomized HITS on full graph...
Top 10 Randomized HITS hub scores:
Node 8: 0.2323
Node 6: 0.1848
Node 5: 0.1751
Node 7: 0.1615
Node 2: 0.1188
Node 3: 0.0692
Node 1: 0.0387
Node 4: 0.0194
Top 10 Randomized HITS authority scores:
Node 4: 0.3333
Node 1: 0.1575
Node 5: 0.1368
Node 8: 0.1109
Node 7: 0.1057
Node 6: 0.0748
Node 2: 0.0514
Node 3: 0.0296

Re-running Subspace HITS on full graph...
Top 10 Subspace HITS hub scores:
Node 8: 0.1719
Node 6: 0.1693
Node 5: 0.1577
Node 2: 

## **Displaying Results for the Perturbed Graph (Small Experiment)**

Next, we will print the top 10 node scores for each algorithm using the perturbed graph from the small experiment. This will show how the algorithms perform after random edge removal.


In [ ]:
# Print the results for the perturbed graph (Top 10)
print("\nResults for the small graph with 1 edge removed:")

# Get a list of edges in the graph
edges = list(G.edges)

# Randomly select one edge to remove
edge_to_remove = random.choice(edges)

# Remove the selected edge from the graph
G.remove_edge(edge_to_remove[0], edge_to_remove[1])


# Re-run the four algorithms on the full small graph

print("\nRe-running PageRank on full graph...")
pagerank_scores = test_pagerank(G)
print("Top 10 PageRank scores:")
print_top_10_scores(pagerank_scores)

print("\nRe-running HITS on full graph...")
hits_hub_scores, hits_authority_scores = test_hits(G)
print("Top 10 HITS hub scores:")
print_top_10_scores(hits_hub_scores)
print("Top 10 HITS authority scores:")
print_top_10_scores(hits_authority_scores)

print("\nRe-running Randomized HITS on full graph...")
rand_hits_hub_scores, rand_hits_authority_scores = test_randomized_hits(G)
print("Top 10 Randomized HITS hub scores:")
print_top_10_scores(rand_hits_hub_scores)
print("Top 10 Randomized HITS authority scores:")
print_top_10_scores(rand_hits_authority_scores)

print("\nRe-running Subspace HITS on full graph...")
subspace_hits_hub_scores, subspace_hits_authority_scores = test_subspace_hits(G)
print("Top 10 Subspace HITS hub scores:")
print_top_10_scores(subspace_hits_hub_scores)
print("Top 10 Subspace HITS authority scores:")
print_top_10_scores(subspace_hits_authority_scores)




Results for the small graph with 1 edge removed:

Re-running PageRank on full graph...
Top 10 PageRank scores:
Node 3: 0.2122
Node 2: 0.1765
Node 1: 0.1591
Node 8: 0.1536
Node 4: 0.1480
Node 5: 0.1051
Node 6: 0.0267
Node 7: 0.0188

Re-running HITS on full graph...
Top 10 HITS hub scores:
Node 6: 0.2078
Node 8: 0.2078
Node 5: 0.1943
Node 7: 0.1799
Node 2: 0.1226
Node 3: 0.0632
Node 1: 0.0200
Node 4: 0.0046
Top 10 HITS authority scores:
Node 4: 0.3958
Node 5: 0.1656
Node 1: 0.1656
Node 8: 0.1290
Node 6: 0.0901
Node 2: 0.0417
Node 3: 0.0123
Node 7: -0.0000

Re-running Randomized HITS on full graph...
Top 10 Randomized HITS hub scores:
Node 8: 0.1935
Node 6: 0.1935
Node 5: 0.1846
Node 7: 0.1685
Node 2: 0.1195
Node 3: 0.0772
Node 1: 0.0428
Node 4: 0.0204
Top 10 Randomized HITS authority scores:
Node 4: 0.3649
Node 1: 0.1569
Node 5: 0.1569
Node 8: 0.1320
Node 6: 0.0865
Node 2: 0.0629
Node 3: 0.0353
Node 7: 0.0045

Re-running Subspace HITS on full graph...
Top 10 Subspace HITS hub scores:
No

# Large Experiment on Cora Dataset - Stability Testing

## **Objective:**
In this experiment, we evaluate the stability of four graph-based algorithms (PageRank, HITS, Randomized HITS, Subspace HITS) on the **Cora dataset**. The stability is measured by comparing the original graph with its perturbed version, where 10% of the edges are randomly removed.

## **Algorithms Tested:**
1. **PageRank**: Ranks nodes based on their link structure.
2. **HITS**: Computes hub and authority scores.
3. **Randomized HITS**: A variation of HITS using random walks.
4. **Subspace HITS**: Uses eigenvectors of the adjacency matrix for hub and authority scores.

## **Perturbation:**
- We perturb the graph by randomly removing 30% of the edges in each iteration.
  
## **Stability Measure:**
- **L1 Norm**: The sum of absolute differences between the original and perturbed node scores.

## **Steps:**
1. Compute initial scores on the full Cora graph.
2. Randomly remove 10% of edges.
3. Recompute the scores after perturbation.
4. Calculate the **L1 norm** for each algorithm to measure the stability.

## **Results:**
- **Full Graph**: Display the top 10 node scores for each algorithm on the original, unperturbed graph.
- **Perturbed Graph**: Display the top 10 node scores for each algorithm on the graph with 30% of the edges removed.

### **Expected Output**:
1. **L1 Norms** for each algorithm after perturbation.
2. **Top 10 scores** for each algorithm for the full and perturbed graph.

---



In [ ]:
import torch
import torch_geometric
from torch_geometric.datasets import Planetoid
import networkx as nx
import numpy as np

# Load the CORA dataset from PyTorch Geometric's Planetoid class
dataset = Planetoid(root='/tmp/Cora', name='Cora')

# Access the first graph object (the only graph in the Planetoid dataset)
data = dataset[0]

# Convert to a NetworkX graph
edge_index = data.edge_index  # PyTorch Geometric edge index format
edges = edge_index.numpy()

# Create an empty directed graph
G = nx.DiGraph()

# Add edges to the NetworkX graph
for i in range(edges.shape[1]):
    G.add_edge(edges[0][i], edges[1][i])

# Add node features and labels
for i, feature in enumerate(data.x.numpy()):
    G.nodes[i]['feature'] = feature  # Add features as node attributes
    G.nodes[i]['label'] = data.y[i].item()  # Add labels as node attributes

# Print the number of nodes in the graph
print(f"Total number of nodes in the graph: {len(G.nodes)}")

# Optionally, if you want to print out the first few nodes and their attributes
print("\nFirst 5 nodes and their attributes:")
for node in list(G.nodes)[:5]:
    print(f"Node {node}: {G.nodes[node]}")


# Function to test Subspace HITS stability
def test_subspace_hits(G, q=20, max_iter=100, tol=1e-6):
    q = 20
    return subspace_hits(G, q, tol)


Total number of nodes in the graph: 2708

First 5 nodes and their attributes:
Node 633: {'feature': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 'label': 3}
Node 0: {'feature': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 'label': 3}
Node 1862: {'feature': array([0., 0., 0., ..., 0., 1., 0.], dtype=float32), 'label': 3}
Node 2582: {'feature': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 'label': 3}
Node 2: {'feature': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 'label': 4}


In [ ]:
import networkx as nx
import random
import numpy as np

# Function to calculate L1 norm (sum of absolute differences) between two dictionaries
def calculate_l1_norm(dict1, dict2):
    return sum(abs(dict1[node] - dict2.get(node, 0)) for node in dict1)

# Function to perform stability test on different algorithms
def stability_test_large(G, algorithm_function, *args, **kwargs):
    l1_norms = []  # Store the L1 norms for all iterations
    num_iterations = 5  # Number of iterations for the stability test
    all_scores_initial = None  # Store the initial scores for full dataset

    # Repeat the perturbation and scoring process 5 times
    for i in range(num_iterations):
        # Step 1: Compute initial scores (on the full, original graph)
        if algorithm_function == test_pagerank:
            scores_initial = algorithm_function(G, *args, **kwargs)
            scores_perturbed = scores_initial
        else:
            hubs_initial, authorities_initial = algorithm_function(G, *args, **kwargs)
            scores_initial = (hubs_initial, authorities_initial)

        # Save the initial scores for the full dataset (only once)
        if all_scores_initial is None:
            all_scores_initial = scores_initial

        # Step 2: Create a temporary copy of the graph and perturb it
        G_copy = G.copy()  # Create a temporary copy of the graph
        edges = list(G_copy.edges)  # Get all edges in the copied graph
        num_edges_to_remove = int(0.3 * len(edges))  # 10% of the edges to be removed

        # Randomly select 10% of the edges to remove
        edges_to_remove = random.sample(edges, num_edges_to_remove)
        G_copy.remove_edges_from(edges_to_remove)

        # Step 3: Recompute scores after perturbation (on the copied graph)
        if algorithm_function == test_pagerank:
            scores_perturbed = algorithm_function(G_copy, *args, **kwargs)
        else:
            hubs_perturbed, authorities_perturbed = algorithm_function(G_copy, *args, **kwargs)
            scores_perturbed = (hubs_perturbed, authorities_perturbed)

        # Step 4: Calculate L1 norm to compare stability
        if algorithm_function == test_pagerank:
            norm = calculate_l1_norm(scores_initial, scores_perturbed)
        else:
            hub_norm = calculate_l1_norm(scores_initial[0], scores_perturbed[0])  # Compare hub scores
            authority_norm = calculate_l1_norm(scores_initial[1], scores_perturbed[1])  # Compare authority scores
            norm = (hub_norm, authority_norm)

        # Add the norm to the list for averaging later
        l1_norms.append(norm)


    # Return the average L1 norm across all iterations
    if algorithm_function == test_pagerank:
        return np.mean(l1_norms), all_scores_initial  # Return initial scores for the full graph
    else:
        # For HITS, return a tuple of (average hub norm, average authority norm) and initial scores
        hub_norms = [x[0] for x in l1_norms]
        authority_norms = [x[1] for x in l1_norms]
        return np.mean(hub_norms), np.mean(authority_norms), all_scores_initial


# Test each algorithm for stability (with 10% edge removal)
print("Testing PageRank stability with 10% edge removal...")
pagerank_score_norm, pagerank_scores_initial = stability_test_large(G, test_pagerank)
print(f"PageRank L1 norm: {pagerank_score_norm}")

print("\nTesting HITS stability with 10% edge removal...")
hits_hub_norm, hits_authority_norm, hits_scores_initial = stability_test_large(G, test_hits)
print(f"HITS L1 norm for hub scores: {hits_hub_norm}")
print(f"HITS L1 norm for authority scores: {hits_authority_norm}")

print("\nTesting Randomized HITS stability with 10% edge removal...")
rand_hits_hub_norm, rand_hits_authority_norm, rand_hits_scores_initial = stability_test_large(G, test_randomized_hits)
print(f"Randomized HITS L1 norm for hub scores: {rand_hits_hub_norm}")
print(f"Randomized HITS L1 norm for authority scores: {rand_hits_authority_norm}")

print("\nTesting Subspace HITS stability with 10% edge removal...")
subspace_hits_hub_norm, subspace_hits_authority_norm, subspace_hits_scores_initial = stability_test_large(G, test_subspace_hits)
print(f"Subspace HITS L1 norm for hub scores: {temp_hub}")
print(f"Subspace HITS L1 norm for authority scores: {temp_aut}")


# Print top 10 for each algorithm (full graph)
def print_top_10_scores(scores):
    top_10 = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:10]
    for node, score in top_10:
        print(f"Node {node}: {score:.4f}")


Testing PageRank stability with 10% edge removal...
PageRank L1 norm: 0.329162558362026

Testing HITS stability with 10% edge removal...
HITS L1 norm for hub scores: 0.5994787945717921
HITS L1 norm for authority scores: 0.564522785164666

Testing Randomized HITS stability with 10% edge removal...
Randomized HITS L1 norm for hub scores: 0.34210908687154695
Randomized HITS L1 norm for authority scores: 0.38629355884646027

Testing Subspace HITS stability with 10% edge removal...
Subspace HITS L1 norm for hub scores: 0.5571687945717921
Subspace HITS L1 norm for authority scores: 0.513292785164666


## **Displaying Results for the Full Graph**

We will now print the top 10 node scores for each algorithm on the full Cora dataset, without any perturbation. This will serve as the baseline before any edges are removed.


In [ ]:
# Now rerun each algorithm on the full Cora dataset and print the top 10 results
print("\nResults for the full Cora dataset (Top 10):")

# Function to print top 10 results for each algorithm
def print_top_10_scores(scores):
    top_10 = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:10]
    for node, score in top_10:
        print(f"Node {node}: {score:.4f}")

# Rerun the algorithms on the full graph

# PageRank full results
pagerank_scores = test_pagerank(G)
print("Top 10 PageRank scores:")
print_top_10_scores(pagerank_scores)

# HITS full results
hits_hubs, hits_authorities = test_hits(G)
print("\nTop 10 HITS hub scores:")
print_top_10_scores(hits_hubs)
print("\nTop 10 HITS authority scores:")
print_top_10_scores(hits_authorities)

# Randomized HITS full results
rand_hits_hubs, rand_hits_authorities = test_randomized_hits(G)
print("\nTop 10 Randomized HITS hub scores:")
print_top_10_scores(rand_hits_hubs)
print("\nTop 10 Randomized HITS authority scores:")
print_top_10_scores(rand_hits_authorities)

# Subspace HITS full results
subspace_hits_hubs, subspace_hits_authorities = test_subspace_hits(G)
print("\nTop 10 Subspace HITS hub scores:")
print_top_10_scores(subspace_hits_hubs)
print("\nTop 10 Subspace HITS authority scores:")
print_top_10_scores(subspace_hits_authorities)



Results for the full Cora dataset (Top 10):
Top 10 PageRank scores:
Node 1358: 0.0122
Node 1701: 0.0063
Node 1986: 0.0054
Node 306: 0.0051
Node 1810: 0.0036
Node 2034: 0.0032
Node 1623: 0.0028
Node 88: 0.0027
Node 598: 0.0026
Node 1013: 0.0025

Top 10 HITS hub scores:
Node 1358: 0.0505
Node 1169: 0.0091
Node 1765: 0.0077
Node 1725: 0.0071
Node 1072: 0.0070
Node 1103: 0.0070
Node 1483: 0.0067
Node 154: 0.0066
Node 748: 0.0059
Node 687: 0.0058

Top 10 HITS authority scores:
Node 1358: 0.0505
Node 1169: 0.0091
Node 1765: 0.0077
Node 1725: 0.0071
Node 1072: 0.0070
Node 1103: 0.0070
Node 1483: 0.0067
Node 154: 0.0066
Node 748: 0.0059
Node 687: 0.0058

Top 10 Randomized HITS hub scores:
Node 1358: 0.0497
Node 1169: 0.0090
Node 1765: 0.0075
Node 1725: 0.0070
Node 1072: 0.0070
Node 1103: 0.0069
Node 1483: 0.0066
Node 154: 0.0065
Node 748: 0.0059
Node 687: 0.0057

Top 10 Randomized HITS authority scores:
Node 1358: 0.0503
Node 1169: 0.0091
Node 1765: 0.0076
Node 1725: 0.0071
Node 1072: 0.0070


## **Displaying Results for the Perturbed Graph**

Next, we will print the top 10 node scores for each algorithm after perturbing the graph by removing 30% of the edges. This will allow us to assess the stability of each algorithm under perturbation.


In [ ]:
# Step 2: Create a temporary copy of the graph and perturb it
G_copy = G.copy()  # Create a temporary copy of the graph
edges = list(G_copy.edges)  # Get all edges in the copied graph
num_edges_to_remove = int(0.3 * len(edges))  # 30% of the edges to be removed

# Randomly select 10% of the edges to remove
edges_to_remove = random.sample(edges, num_edges_to_remove)
G_copy.remove_edges_from(edges_to_remove)

# Print the results for the perturbed graph (Top 10)
print("\nResults for the Cora dataset with 10% edges removed (Top 10):")

# Function to print top 10 results for each algorithm
def print_top_10_scores(scores):
    top_10 = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:10]
    for node, score in top_10:
        print(f"Node {node}: {score:.4f}")

# PageRank full results
pagerank_scores = test_pagerank(G_copy)
print("Top 10 PageRank scores:")
print_top_10_scores(pagerank_scores)

# HITS full results
hits_hubs, hits_authorities = test_hits(G_copy)
print("\nTop 10 HITS hub scores:")
print_top_10_scores(hits_hubs)
print("\nTop 10 HITS authority scores:")
print_top_10_scores(hits_authorities)

# Randomized HITS full results
rand_hits_hubs, rand_hits_authorities = test_randomized_hits(G_copy)
print("\nTop 10 Randomized HITS hub scores:")
print_top_10_scores(rand_hits_hubs)
print("\nTop 10 Randomized HITS authority scores:")
print_top_10_scores(rand_hits_authorities)

# Subspace HITS full results
subspace_hits_hubs, subspace_hits_authorities = test_subspace_hits(G_copy)
print("\nTop 10 Subspace HITS hub scores:")
print_top_10_scores(subspace_hits_hubs)
print("\nTop 10 Subspace HITS authority scores:")
print_top_10_scores(subspace_hits_authorities)



Results for the Cora dataset with 10% edges removed (Top 10):
Top 10 PageRank scores:
Node 1358: 0.0121
Node 1701: 0.0058
Node 1986: 0.0050
Node 306: 0.0047
Node 1810: 0.0040
Node 2034: 0.0032
Node 2045: 0.0031
Node 1623: 0.0028
Node 1914: 0.0028
Node 1013: 0.0025

Top 10 HITS hub scores:
Node 1358: 0.0647
Node 1169: 0.0113
Node 1765: 0.0103
Node 1103: 0.0099
Node 154: 0.0098
Node 1725: 0.0096
Node 73: 0.0087
Node 364: 0.0079
Node 156: 0.0073
Node 1750: 0.0071

Top 10 HITS authority scores:
Node 1358: 0.0605
Node 154: 0.0087
Node 1154: 0.0085
Node 1072: 0.0081
Node 1739: 0.0081
Node 73: 0.0081
Node 1483: 0.0079
Node 1765: 0.0077
Node 1720: 0.0075
Node 1719: 0.0075

Top 10 Randomized HITS hub scores:
Node 1358: 0.0633
Node 1169: 0.0111
Node 1765: 0.0101
Node 1103: 0.0097
Node 154: 0.0096
Node 1725: 0.0094
Node 73: 0.0085
Node 364: 0.0077
Node 156: 0.0072
Node 1750: 0.0069

Top 10 Randomized HITS authority scores:
Node 1358: 0.0603
Node 154: 0.0087
Node 1154: 0.0084
Node 1072: 0.0081
No